In [ ]:
# Import stuff
import pandas as pd
import numpy as np
import re
import functions

In [ ]:
# Load Data
hotels = pd.read_csv('data/Hotel_Reviews.csv')

In [ ]:
hotels.head()

In [ ]:
# lat and lng have a lot of missing values so they will be dropped
# days_since_review don't matter for much so let's remove it
# words/tags also dont't matter for much
# In short the most important information is:
# 	Hotel_Address, Hotel_Name, Reviewer_Nationality, Positive_Review, Negative_Review, Average_Score, Reviewer_Score
hotels_short = hotels[['Hotel_Address', 'Hotel_Name', 'Reviewer_Nationality', 'Positive_Review', 'Negative_Review', 'Average_Score', 'Reviewer_Score', 'lat', 'lng']]

In [ ]:
hotels_short.dropna(inplace=True)

In [ ]:
hotels_short.isna().sum()

In [ ]:
hotels_short['Country'] = hotels_short['Hotel_Address'].apply(lambda x: functions.get_country(x))

In [ ]:
# Drop reviews with empty nationality 
to_drop = hotels_short[hotels_short['Reviewer_Nationality'] == ' ' ].index

# Delete these row indexes from dataFrame
hotels_short.drop(to_drop , inplace=True)

In [ ]:
final_review_data = hotels_short[['Hotel_Name','Reviewer_Nationality','Reviewer_Score', 'Country','Average_Score','lat','lng']]

In [ ]:
# Trim Reviewer_Nationality
final_review_data['Reviewer_Nationality'] = final_review_data['Reviewer_Nationality'].apply(lambda x: x.strip())

In [ ]:
final_review_data.rename(columns={
    'Hotel_Name' : 'name',
    'Reviewer_Nationality': 'reviewer_country',
    'Reviewer_Score': 'reviewer_score', 
    'Country': 'review_country',
    'Average_Score': 'review_average',
    'lng': 'long'
}, inplace=True)

In [ ]:
%%time
# Get unique countries and create a dict to rename values in cols for countries

countries = list(final_review_data['reviewer_country'].value_counts().keys())
country_trans_dict = {c:functions.get_country_details(c)['idCountry'] for c in countries}


In [ ]:
final_review_data['reviewer_country'].replace(country_trans_dict, inplace=True)
final_review_data['review_country'].replace(country_trans_dict, inplace=True)

In [ ]:
final_review_data['review_category'] = '1' # for Hotels

In [ ]:
final_review_data

In [ ]:
for i, r in final_review_data.iterrows():
    q = "INSERT INTO `final_project`.`reviews` (`name`, `reviewer_score`, `reviewer_country`, `review_country`, `review_average`, `review_category`, `lat`,`long`) VALUES (%s,%s,%s,%s,%s,%s,%s,%s)"
    val = (r['name'],r['reviewer_score'],r['reviewer_country'],r['review_country'],r['review_average'],r['review_category'],r['lat'],r[ 'long'])
    insert = functions.db.insert(q,val)
    print(i, "out of", len(final_review_data),"Inserted:", insert)
    break